In [4]:
%load_ext autoreload
%autoreload 2
import torch
from torch import nn
from mvn.utils import cfg
from train import setup_human36m_dataloaders
import matplotlib.pyplot as plt
from mvn.datasets import utils as dataset_utils

from mvn.models.v2v import V2VModelAdaIN, Basic3DBlockAdaIN, Res3DBlockAdaIN, Upsample3DBlockAdaIN
from mvn.models.volumetric_temporal import Seq2VecModel, VolumetricAdaINConditionedTemporalNet

device = torch.cuda.current_device()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# l1 = [16, 32,32,32, 32,32, 32,32]

# l2 = [32,32, 64,64,64, 64,64, 128,128,128, 128,128, 128,128, \
#         128,128, 128,128, 128,128, 128,128, 128,128, 128,128, 128, \
#         128,128, 128, 128,128, 128, 128,128, 64, 64,64, 32]
# l3 = [32,32, 32, 32]

In [6]:
config_path = 'experiments/human36m/train/human36m_vol_temporal_softmax.yaml'
config = cfg.load_config(config_path)

train_loader, val_loader, _ = setup_human36m_dataloaders(config, is_train=True,distributed_train=False)

In [7]:
for batch in train_loader:
    break

In [8]:
model = VolumetricAdaINConditionedTemporalNet(config).cuda()

Loading pretrained weights from: ./data/pose_resnet_4.5_pixels_human36m.pth
Parameters [{'final_layer.bias', 'final_layer.weight'}] were not inited
Successfully loaded pretrained weights for backbone
backbone inited 2.5086
v2v inited 0.1534
Seq2VecModel inited 0.0144
affine_mappings inited 0.0301


In [9]:
images_batch, keypoints_3d_batch_gt, keypoints_3d_validity_batch_gt, proj_matricies_batch = dataset_utils.prepare_batch(batch, device, config)

print(images_batch.shape, keypoints_3d_batch_gt.shape, proj_matricies_batch.shape)

torch.Size([1, 4, 3, 384, 384]) torch.Size([1, 17, 3]) torch.Size([1, 4, 3, 4])


In [10]:
(vol_keypoints_3d,
    features,
    volumes,
    vol_confidences,
    cuboids,
    coord_volumes,
    base_points) = model(images_batch, batch)

